In [1]:
!python3 -m venv qml-env
!source qml-env/bin/activate  # on Windows: .\qml-env\Scripts\activate



In [2]:
!pip install pennylane


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 80.1 MB/s eta 0:00:00:00:0100:01


In [41]:
!pip show pennylane-lightning


Name: PennyLane_Lightning
Version: 0.40.0
Summary: PennyLane-Lightning plugin
Home-page: 
Author: 
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: pennylane, scipy-openblas32
Required-by: PennyLane


In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 10.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0


In [4]:
pip install tensorflow==2.12
pip install tensorflow-quantum


SyntaxError: invalid syntax (1131985662.py, line 1)

In [7]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import optuna
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from torch.utils.data import TensorDataset, DataLoader

# Use GPU if available.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -----------------------
# 1. DATASET PREPARATION
# -----------------------

# Load the Adult dataset from OpenML.
data = fetch_openml("adult", version=2, as_frame=True)
df = data.frame

# Select only numeric features.
numeric_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
X = df[numeric_cols].to_numpy()
y = df['class'].to_numpy()
# Convert target to binary: 0 for "<=50K", 1 for ">50K".
y = np.where(y == '<=50K', 0, 1)

# For demonstration, use only 10% of the samples.
np.random.seed(42)
indices = np.random.choice(X.shape[0], size=int(0.1 * X.shape[0]), replace=False)
X = X[indices]
y = y[indices]
print("Number of samples used (10% subset):", X.shape[0])

# Reduce features to 2 using PCA (for our 2-qubit circuit).
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Scale features to [-π, π].
scaler = MinMaxScaler(feature_range=(-np.pi, np.pi))
X_scaled = scaler.fit_transform(X_pca)

# Split into train+validation (80%) and test (20%),
# then further split train+validation into training (60%) and validation (20%).
X_train_val_np, X_test_np, y_train_val_np, y_test_np = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)
X_train_np, X_val_np, y_train_np, y_val_np = train_test_split(
    X_train_val_np, y_train_val_np, test_size=0.25, random_state=42, stratify=y_train_val_np
)

# Convert to PyTorch tensors and move to device.
X_train = torch.tensor(X_train_np, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train_np, dtype=torch.float32).unsqueeze(1).to(device)
X_val = torch.tensor(X_val_np, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val_np, dtype=torch.float32).unsqueeze(1).to(device)
X_test = torch.tensor(X_test_np, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test_np, dtype=torch.float32).unsqueeze(1).to(device)

# Optionally, create a DataLoader for tuning.
batch_size = 256
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# ----------------------------------------
# 2. CLASSICAL SURROGATE MODEL FOR TUNING
# ----------------------------------------

# We define a fast classical model (a simple linear model) to tune hyperparameters.
class ClassicalSurrogate(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)  # directly mapping from the 2 PCA features to a logit
    
    def forward(self, x):
        return self.linear(x)

def objective(trial):
    # Suggest hyperparameters.
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-1)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
    n_epochs = trial.suggest_int('n_epochs', 5, 50)  # keep epochs small for fast tuning
    
    model = ClassicalSurrogate().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.BCEWithLogitsLoss()
    
    for epoch in range(n_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
    
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val).item()
    return val_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)
print("Best hyperparameters from surrogate tuning:", study.best_trial.params)

# ----------------------------------------
# 3. QUANTUM CIRCUIT DEFINITION
# ----------------------------------------

# Optionally, check for PennyLane-Lightning:
# Run `pip show pennylane-lightning` in your terminal.
# If not installed, you can install via: pip install pennylane-lightning
# Here we switch to the Lightning simulator for potentially faster simulation:
dev = qml.device("lightning.qubit", wires=2)
observable = qml.Hamiltonian([1, 1], [qml.PauliZ(0), qml.PauliZ(1)])

@qml.qnode(dev, interface="torch")
def quantum_layer_q(inputs, weights):
    if inputs.ndim == 2:
        inputs = inputs[0]
    qml.RX(inputs[0], wires=0)
    qml.RX(inputs[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(weights[0], wires=0)
    qml.RY(weights[1], wires=1)
    exp_val = qml.expval(observable)
    return qml.math.reshape(exp_val, (1,))

weight_shapes = {"weights": (2,)}
qlayer = qml.qnn.TorchLayer(quantum_layer_q, weight_shapes=weight_shapes)

# ----------------------------------------
# 4. HYBRID QUANTUM-CLASSICAL MODEL
# ----------------------------------------

class HybridQNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.quantum = qlayer  # Expects input shape (1,2) and returns (1,)
        self.linear = nn.Linear(1, 1)
    
    def forward(self, inputs):
        q_out_list = []
        for sample in inputs:
            sample_batched = sample.unsqueeze(0)  # (1,2)
            q_out_sample = self.quantum(sample_batched)  # (1,)
            q_out_list.append(q_out_sample.squeeze(0))   # (1,)
        q_out = torch.stack(q_out_list)
        if q_out.ndim == 1:
            q_out = q_out.unsqueeze(1)
        out = self.linear(q_out)
        return out

# ----------------------------------------
# 5. RETRAIN FINAL MODEL & EVALUATION
# ----------------------------------------

# Combine training and validation sets.
X_train_full = torch.cat((X_train, X_val), dim=0)
y_train_full = torch.cat((y_train, y_val), dim=0)

best_params = study.best_trial.params
final_model = HybridQNN().to(device)
final_optimizer = optim.Adam(final_model.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'])
criterion = nn.BCEWithLogitsLoss()

n_epochs_final = best_params['n_epochs']
final_dataset = TensorDataset(X_train_full, y_train_full)
final_loader = DataLoader(final_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(n_epochs_final):
    final_model.train()
    for X_batch, y_batch in final_loader:
        final_optimizer.zero_grad()
        outputs = final_model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        final_optimizer.step()

final_model.eval()
with torch.no_grad():
    test_logits = final_model(X_test)
    test_preds = (torch.sigmoid(test_logits) > 0.5).float()
    accuracy = (test_preds.eq(y_test).sum() / y_test.shape[0]).item()
    print("Test Accuracy:", accuracy)


Using device: cuda


/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
[I 2025-04-06 05:48:22,112] A new study created in memory with name: no-name-6f12620b-ff56-4117-aaed-7450a930a5c7


Number of samples used (10% subset): 4884


/tmp/ipykernel_42/313193639.py:84: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-4, 1e-1)
/tmp/ipykernel_42/313193639.py:85: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
[I 2025-04-06 05:48:24,016] Trial 0 finished with value: 0.6140192747116089 and parameters: {'lr': 0.0006968350728123136, 'weight_decay': 1.7389225710840467e-06, 'n_epochs': 41}. Best is trial 0 with value: 0.6140192747116089.
[I 2025-04-06 05:48:25,462] Trial 1 finished with value: 0.5134274959564209 and parameters: {'lr': 0.060892150727319294, 'weight_decay': 0.0005753744964653768, 'n_epochs': 2

Best hyperparameters from surrogate tuning: {'lr': 0.06217367624213928, 'weight_decay': 1.30896899505966e-06, 'n_epochs': 40}
Test Accuracy: 0.7778915166854858
